# Zusammenfügen von Datensätzen in `pandas`

Wir kennen in Pandas nun schon `pd.concat`. Diese Funktion dient zum 
*einfachen* Aneinanderfügen von Tabellen, ohne Berücksichtigung von Spalten,
 die diese gemeinsam haben.

Heute lernen wir, wie wir Tabellen basierend auf übereinstimmenden Spalten 
(z.B. Bestell-ID; Modellnummer; ...) zusammenfügen. Dabei werden anhand der
 Indizes oder anhand einer gemeinsamen Spalte Einträge verbunden, die in 
 beiden Tabellen übereinstimmen. Wir kennen diese "Joins" noch aus Excel 
 mit den Funktionen `SVERWEIS()` und `INDEX(VERGLEICH())` und aus Power 
 Query. In Pandas benutzt man mit die DataFrame-Methoden `join()` und `merge()`.

Hierfür ist es nützlich, sich noch einmal die verschiedenen Arten anzuschauen, auf
  die man Tabellen zusammenfügen kann – die sogenannten Joins.

In [ ]:
import pandas as pd


## Anfügen von Daten: pd.concat


In [ ]:
# Series mit Temperatur-Messwerten:
data = [4.5, 6.3, 3.8, 5.1, 4.9, 5.7, 4.2, 6.0]
temp_series = pd.Series(data, name="Temperatur")
temp_series

In [ ]:
# Series mit Zeitstempeln für Messzeitpunkt:
uhrzeiten = ['2023-01-02 19:08',
             '2023-01-04 18:17',
             '2023-01-06 06:03',
             '2023-01-09 02:17',
             '2023-01-12 22:02',
             '2023-01-17 16:00',
             '2023-01-22 21:04',
             '2023-01-24 11:16']

# Eigentlich besser, man macht datetime-Objekte daraus, aber das Thema kommt erst später dran,
# bitte also Geduld ;)

# So könnte man das tun: pd.Series(pd.to_datetime(uhrzeiten))

time_series = pd.Series(uhrzeiten, name="Zeitstempel")
time_series

#### Beide Serien zu einem DataFrame verbinden


In [ ]:
# temp_series und time_series verbinden
temps_df = pd.concat([time_series, temp_series], axis=1)
temps_df

#### Was, wenn die Indices nicht so gut zusammenspielen?

In [ ]:
# Eigene Indizes vergeben:

time_series_2 = time_series.copy()
# Lücken: 0, 1, 4, 10
time_series_2.index = [2, 3, 5, 6, 7, 8, 9, 11]

temp_series_2 = temp_series.copy()
# Lücken: 1, 3, 9, 11
temp_series_2.index = [0, 2, 4, 5, 6, 7, 8, 10]

print(time_series_2)
print(temp_series_2)


In [ ]:
# Serien mit benannten indizes verbinden
# Erzeugt NaN, wenn Indices nicht in beiden Serien vorkommen
temps_df2 = pd.concat([time_series_2, temp_series_2],
                 axis=1)

temps_df2.sort_index()

In [ ]:
# Index muss in beiden Serien einzigartig sein (Keine Duplikate)!
# Sonst funktioniert concat nicht.

In [ ]:
# Test
temp_series_3 = temp_series.copy()
temp_series_3.index = [2, 3, 5, 6, 7, 7, 9, 11]

time_series_3 = time_series.copy()
time_series_3.index = [0, 2, 2, 5, 6, 7, 8, 10]

In [ ]:
temps_df3 = pd.concat([time_series_3, temp_series_3],
                       axis=1)

temps_df3.sort_index()

## Verbinden über Index-Vergleich


### `DataFrame.join`

Verwendet den Index oder eine bestimmte Spalte des DataFrames, der die Methode aufruft und fügt die Daten übereinstimmender Indizes des anderen DataFrames seitlich an. 

Für weitere Infos: [Link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)

In [ ]:
# Erstellen zweier DataFrames mit
# unterschiedlichen Indices
contacts1 = pd.DataFrame({"Name": ["Franz", "Lena", "Chloé"],
                    "Alter": ["67", "31", "41"]},
                   index=["K0", "K1", "K2"])

contacts2 = pd.DataFrame({"Wohnort": ["Rostock", "Nürnberg", "Berlin"],
                    "Telefonnummer": ["030 215783", "030 847735", "030 781404"]},
                   index=["K0", "K2", "K3"])

print(contacts1)
print()
print(contacts2)
# Was stellen wir an den DataFrames fest?

In [ ]:
# pd.concat kann zwar auch joins, aber nur inner 
# oder outer join (kein left oder right join)
# Standardverhalten ist übrigens: outer und erzeugt potentiell NaNs:
pd.concat([contacts1, contacts2], axis=1, join="outer")

In [ ]:
pd.concat([contacts1, contacts2], axis=1, join="inner")

#### DataFrame.join() ermöglicht weitere Joins

In [ ]:
# Left Join (Standardverhalten von join!)
# Alle Keys aus dem ERSTEN (linken) Datensatz werden genutzt
# und um Daten aus dem andern (rechten) Datensatz ergänzt.
# An Indexpositionen, über die der rechte Datensatz nicht verfügt, entstehen NaNs:
contacts1.join(contacts2)

In [ ]:
# Von contacts2 kommend entstehen die Lücken an anderen Stellen, wo eben contacts1 keine Indices hat:
contacts2.join(contacts1)

In [ ]:
# Right join
# Keys der rechten (zweiten) Datensatzes werden genutzt,
# und um entsprechende Daten aus dem linken (ersten) ergänzt.
# Wo der erste keine Indices hat, entstehen NaNs 
contacts1.join(contacts2, how="right")

In [ ]:
# Outer join
# Alle Keys aus BEIDEN Datensätzen werden genutzt
# Maximale "NaN-Dichte" wird erreicht:
contacts1.join(contacts2, how="outer")

In [ ]:
# Inner join
# Nur Keys, die in BEIDEN Datensätzen vorhanden sind
# Es kommt zu keinen NaN-Werten (ist unmöglich!):
contacts1.join(contacts2, how="inner")

In [ ]:
# Cross join
# Erzeugt eine Kombination jeder Zeile des ersten Datensatzes
# mit jeder Zeile des zweiten Datensatzes (hier nicht gerade sinnvoll)
contacts1.join(contacts2, how="cross")

#### Zusammenfügen mehrerer Datensätze

In [ ]:
print(contacts1)
print()
print(contacts2)


In [ ]:
# Mit zwei dfs kennen wir das Spiel schon:
contacts1.join(contacts2, how='outer')

In [ ]:
# Aber jetzt haben wir noch eine Nummer 3:
contacts3 = pd.DataFrame({"Position": ["Rentner", "Verkäuferin", "Data Engineer"],
                    "Gehalt": ["1400", "3000", "3800"]},
                   index=["K1", "K3", "K4"])

contacts3

In [ ]:
# joinen mehrerer dfs an contacts1 über Liste möglich:
contacts1.join([contacts2, contacts3], how="outer")

In [ ]:
# Bonusfrage: Was zum Teufel ist denn hier passiert?
contacts1.join([contacts2, contacts3], how="inner")

In [ ]:
# Bonus-Info!
# join hat noch eine weitere Fähigkeit mit 'on'
# Der join kann über eine wählbare Spalte aus dem linken DataFrame 
# mit dem Index des rechten DataFrames erfolgen.
# Wir modifizieren contacts1, sodass dort die "Indices" in einer Spalte vorkommen!

contacts1 = pd.DataFrame({"Name": ["Franz", "Lena", "Chloé"],
                          "Alter": ["67", "31", "41"],
                          "Kontakt-ID": ["K0", "K1", "K2"]})

contacts2 = pd.DataFrame({"Wohnort": ["Rostock", "Nürnberg", "Berlin"],
                          "Telefonnummer": ["030 215783", "030 847735", "030 781404"]}, 
                          index=["K0", "K2", "K3"])

print(contacts1)
print()
print(contacts2)

In [ ]:
contacts1.join(contacts2, on='Kontakt-ID')

In [ ]:
# Aber man kann solche Dinge (und noch mehr) auch mit merge erreichen!

#### Beide Serien zu einem DataFrame verbinden


#### Übungsaufgabe `concat` + `join`

Zeit: 30 Minuten

Gegeben sind Temperaturmessdaten (`temp`), Zeitstempel (`uhrzeiten`), 
 Luftdruckdaten (`druck_dict`) und Geolokationsdaten (`geo_dict`).
1. Wandel die Temperaturdaten und Zeitstempel in Series um und kombiniere 
sie anschließend zu einem DataFrame namens `temp_df`.
2. Wandel die beiden Dictionaries jedes in jeweils einen DataFrame um (`druck_df`, `geo_df`).
3. Füge die Druckdaten an den DataFrame aus 1. an und speichere den neuen 
DataFrame als `df_gesamt`.
4. Kombiniere `df_gesamt` so mit dem Geolokalisation-DataFrame, dass du für 
jede in `df_gesamt` vorkommende Stadt die Breiten- und Längengrade im 
resultierenden DataFrame erhältst. (Tipp: Hierfür müssen die Indices 
verändert werden).

    Output:
    ```
                        Zeitstempel  Temperatur  Luftdruck  Breitengrad  Laengengrad  
    Location                                                              
    Berlin         2023-01-01 19:08         4.5     1001.2        52.31        13.24
    München        2023-01-01 18:17         6.3      997.8        48.80        11.34
    Wilhelmshaven  2023-01-01 06:03         3.8     1002.5          NaN          NaN
    Kassel         2023-01-01 02:17         5.1     1000.1        49.28      -123.13
    Frankfurt      2023-01-01 22:02         4.9      998.9        47.61      -122.33
    Duisburg       2023-01-01 16:00         5.7     1001.5        53.55      -113.49
    Dresden        2023-01-01 21:04         4.2      999.2          NaN          NaN
    Würzburg       2023-01-01 11:16         6.0     1002.8        51.05      -114.07
    ```


In [ ]:
# Temperatur-Messwerte
temp = [4.5, 6.3, 3.8, 5.1, 4.9, 5.7, 4.2, 6.0]

# Zeitstempel für Messzeitpunkt
uhrzeiten = ['2023-01-01 19:08',
             '2023-01-01 18:17',
             '2023-01-01 06:03',
             '2023-01-01 02:17',
             '2023-01-01 22:02',
             '2023-01-01 16:00',
             '2023-01-01 21:04',
             '2023-01-01 11:16']

# Orte und Luftdruckmessung
druck_dict = {'Location': ['Berlin', 'München', 'Wilhelmshaven',
                           'Kassel', 'Frankfurt', 'Duisburg',
                           'Dresden', 'Würzburg'],
              'Luftdruck': [1001.2, 997.8, 1002.5, 1000.1, 998.9,
                            1001.5, 999.2, 1002.8]}

# Längen- und Breitengrade der Orte
geo_dict = {'Location': ['Berlin', 'München', 'Hamburg', 'Köln',
                         'Frankfurt', 'Duisburg', 'Kassel', 'Würzburg'],
            'Breitengrad': [52.31, 48.8, 53.33, 45.75, 47.61,
                            53.55, 49.28, 51.05],
            'Laengengrad': [13.24, 11.34, 10.0, -122.43, -122.33,
                            -113.49, -123.13, -114.07]}

In [ ]:
# ENDE ÜBUNG

### `pandas.merge` (Datensätze zusammenfügen über Index oder / und Column)

Bei `pd.merge` können wir ein `on=` Keyword angeben, wodurch wir Tabellen 
auch über normale Spalten statt über den Index zusammenführen können. Hier 
müssen nicht einmal die Spaltennamen zwingend übereinstimmen. Außerdem hat 
`merge` noch viele andere zusätzliche Optionen, die es bei `join` nicht 
gibt, zum Beispiel die Benutzung mehrerer Schlüsselspalten.

Es gibt sogar eine `merge_asof` Funktion, welche auch ungenaue 
Übereinstimmungen erlaubt, ähnlich wie der optionale Parameter 
Bereich_Verweis in Excels SVERWEIS, wo eine ungenaue Übereinstimmung über 
"WAHR" festgelegt werden konnte.
Jedoch gibt es auch hier in Pandas wieder viel mehr Einstellungsmöglichkeiten.

Mehr Information: [Link](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)

In [ ]:
# DataFrame aus Übung erstellen vor dem Zusammenführen:
df = pd.concat([time_series, temp_series, druck_df],
               axis=1)
df

In [ ]:
# Geo-Dataframe soll gänzlich anderen Index haben.
# Vorarbeit:
geo_length = len(geo_dict['Location'])

In [ ]:
geo_df = pd.DataFrame(geo_dict, index=[f'Eintrag {i}' for i in range (geo_length)])

In [ ]:
geo_df

In [ ]:
# Zusammenführen beider dfs jetzt durch merge,
# OHNE dass Indices passen:
df.merge(geo_df, on="Location")
# how ist standardmäßig auf 'inner' gesetzt

In [ ]:
# Achtung: andere defaults bei .merge() als bei .join()
# bei .merge() ist inner join default (bei .join() ist es left)
df.merge(geo_df, on="Location", how="left")

#### merge() erlaubt uns auch das Verbinden von unterschiedlich bezeichneten Spalten

In [ ]:
# Umbennenen der Spalte Location von geo_df in Stadt
geo_df.rename(columns={"Location": "Stadt"}, inplace=True)
geo_df

In [ ]:
# Verwenden von right_on und left_on, um unterschiedliche
# Spaltennamen zu mergen
df.merge(geo_df, left_on="Location", right_on="Stadt", how="left")

#### Übungsaufgabe `merge`

Nachfolgender Dictionaries enthalten Daten zu Kunden und Produktkäufen.
Deine Aufgabe ist es, daraus zwei DataFrames zu erstellen und danach die beiden DataFrames so mitttels merge zu verbinden, dass zu allen Produktdaten die entsprechenden Kundendaten erscheinen, soweit verfügbar (ansonsten NaN-Werte).

In [ ]:
customer_data = {
    'CustomerID': [101, 102, 103, 104, 105, 106, 107],
    'CustomerName': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace'],
    'Email': ['alice@mail.com', 'bob@mail.com', 'charlie@mail.com', 'david@mail.com', 'eva@mail.com', 'frank@mail.com', 'grace@mail.com'],
    'JoinDate': ['2022-05-01', '2021-06-15', '2020-08-20', '2022-11-25', '2023-01-05', '2021-09-10', '2020-12-31']
}

purchase_data = {
    'ClientID': [101, 102, 103, 108, 105, 106, 107, 102],
    'ProductID': [201, 202, 203, 204, 205, 206, 207, 205],
    'PurchaseDate': ['2023-01-10', '2023-02-15', '2023-01-20', '2023-03-10', '2023-01-30', '2023-03-05', '2023-01-25', '2023-04-01'],
}